# Script for a period

In [1]:
import requests
import json
import pandas as pd
from clickhouse_connect import get_client
from datetime import datetime, timedelta, date
from dotenv import load_dotenv
import os
import time

load_dotenv()

True

In [2]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [3]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [4]:
def process_advert_data(data):
    # Flatten the JSON data
    df = pd.json_normalize(
        data['adverts'],
        record_path='advert_list', 
        meta=['type', 'status', 'count']
    )
    
    # Convert 'changeTime' to datetime format
    df['changeTime'] = pd.to_datetime(df['changeTime'])
    
    # Reset the index
    df = df.reset_index(drop=True)
    
    return df

# Now you can process each dataset with one line:
df_guten = process_advert_data(data_guten)
df_giper = process_advert_data(data_giper)
df_kitchen = process_advert_data(data_kitchen)
df_smart = process_advert_data(data_smart)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [5]:
# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(df, headers, project_name, chunk_size=50):
    chunks = [df['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df), chunk_size)]
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    
    all_campaign_data = pd.DataFrame(all_campaign_data)
    all_campaign_data = all_campaign_data.sort_values(by='createTime', ascending=False)
    all_campaign_data['Project'] = project_name
    all_campaign_data['Marketplace'] = 'Wildberries'
    
    return all_campaign_data

# Fetch campaign data for each project
campaigns_guten = fetch_campaign_data(df_guten, headers_guten, 'WB-GutenTech')
campaigns_giper = fetch_campaign_data(df_giper, headers_giper, 'WB-ГиперМаркет')
campaigns_kitchen = fetch_campaign_data(df_kitchen, headers_kitchen, 'WB-KitchenAid')
campaigns_smart = fetch_campaign_data(df_smart, headers_smart, 'WB-Smart-Market')

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

print("Combined Campaign Data")
#campaigns_kitchen

Data retrieved successfully for WB-GutenTech chunk 1
Data retrieved successfully for WB-GutenTech chunk 2
Data retrieved successfully for WB-GutenTech chunk 3
Data retrieved successfully for WB-GutenTech chunk 4
Data retrieved successfully for WB-GutenTech chunk 5
Data retrieved successfully for WB-GutenTech chunk 6
Data retrieved successfully for WB-GutenTech chunk 7
Data retrieved successfully for WB-GutenTech chunk 8
Data retrieved successfully for WB-GutenTech chunk 9
Data retrieved successfully for WB-GutenTech chunk 10
Data retrieved successfully for WB-ГиперМаркет chunk 1
Data retrieved successfully for WB-ГиперМаркет chunk 2
Data retrieved successfully for WB-KitchenAid chunk 1
Data retrieved successfully for WB-Smart-Market chunk 1
Combined Campaign Data


In [6]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-05-30,2025-06-02,AND_Тонометр 1100_30.05.2025,25905937,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-05-30,2025-06-02,"AND_Тонометр UB-402, UB-403_30.05.2025",25900784,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-05-23,2025-05-23,GRAEF_Ножеточка CC-120_3%_23.05.2025,25742934,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-05-20,2025-05-20,Oasis_Микроволновка_20.05.2025_7%,25655618,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
4,2025-05-20,2025-05-15,2025-05-20,Oasis_Микроволновка_15.05.2025_7%,25538281,Кампания завершена,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
524,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
525,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
526,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
527,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [ ]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"
# Automatically get yesterday's date
yesterday = "2025-05-27"
specific_date = str(yesterday)  # Replace with your desired date

### Guten

In [8]:
def fetch_campaign_stats(campaign_ids, headers, date, base_url,project_name, chunk_size=100):
    chunks = [campaign_ids['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaign_ids), chunk_size)]
    all_data = []
    for idx, chunk in enumerate(chunks): 
        #Construct the payload for the current chunk
        payload = [{"id": campaign_id, "dates": [date]} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(base_url, headers=headers, json=payload)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1} for {project_name}" )
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
            if "no companies with correct intervals" in response.text:
                print("Stopping execution due to invalid interval error")
                break
            
        # Add a delay to avoid hitting API rate limits
        time.sleep(65)
            
    return all_data if all_data else [{}] 

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_stats(campaigns_guten, headers_guten, specific_date, url, "WB-GutenTech")
all_campaign_data_giper = fetch_campaign_stats(campaigns_giper, headers_giper, specific_date, url, "WB-ГиперМаркет")
all_campaign_data_kitchen = fetch_campaign_stats(campaigns_kitchen, headers_kitchen, specific_date, url, "WB-KitchenAid")
all_campaign_data_smart = fetch_campaign_stats(campaigns_smart, headers_smart, specific_date, url, "WB-Smart-Market")


Data retrieved successfully for chunk 1 for WB-GutenTech
Data retrieved successfully for chunk 2 for WB-GutenTech
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Stopping execution due to invalid interval error
Data retrieved successfully for chunk 1 for WB-ГиперМаркет
Data retrieved successfully for chunk 1 for WB-KitchenAid
Data retrieved successfully for chunk 1 for WB-Smart-Market


In [9]:
all_campaign_data_giper

[{'views': 1425,
  'clicks': 61,
  'ctr': 4.28,
  'cpc': 3.37,
  'sum': 205.36,
  'atbs': 15,
  'orders': 11,
  'cr': 18.03,
  'shks': 11,
  'sum_price': 6105,
  'dates': ['2025-05-27'],
  'days': [{'date': '2025-05-27T03:00:00+03:00',
    'views': 1425,
    'clicks': 61,
    'ctr': 0,
    'cpc': 3.37,
    'sum': 205.36,
    'atbs': 15,
    'orders': 11,
    'cr': 18.03,
    'shks': 11,
    'sum_price': 6105,
    'apps': [{'views': 136,
      'clicks': 4,
      'ctr': 2.94,
      'cpc': 4.96,
      'sum': 19.85,
      'atbs': 1,
      'orders': 1,
      'cr': 25,
      'shks': 1,
      'sum_price': 555,
      'nm': [{'views': 136,
        'clicks': 4,
        'ctr': 2.94,
        'cpc': 4.96,
        'sum': 19.85,
        'atbs': 1,
        'orders': 1,
        'cr': 25,
        'shks': 1,
        'sum_price': 555,
        'name': 'Насадка для зубной щетки CrossAction 1 шт',
        'nmId': 232802490}],
      'appType': 1},
     {'views': 561,
      'clicks': 27,
      'ctr': 4.81,
   

In [10]:
all_campaign_data_smart

[{'views': 4671,
  'clicks': 157,
  'ctr': 3.36,
  'cpc': 3.24,
  'sum': 508.56,
  'atbs': 23,
  'orders': 6,
  'cr': 3.82,
  'shks': 6,
  'sum_price': 69716,
  'dates': ['2025-05-27'],
  'days': [{'date': '2025-05-27T03:00:00+03:00',
    'views': 4671,
    'clicks': 157,
    'ctr': 0,
    'cpc': 3.24,
    'sum': 508.56,
    'atbs': 23,
    'orders': 6,
    'cr': 3.82,
    'shks': 6,
    'sum_price': 69716,
    'apps': [{'views': 659,
      'clicks': 10,
      'ctr': 1.52,
      'cpc': 6.75,
      'sum': 67.51,
      'atbs': 3,
      'orders': 0,
      'cr': 0,
      'shks': 0,
      'sum_price': 0,
      'nm': [{'views': 93,
        'clicks': 0,
        'ctr': 0,
        'cpc': 0,
        'sum': 9.74,
        'atbs': 1,
        'orders': 0,
        'cr': 0,
        'shks': 0,
        'sum_price': 0,
        'name': 'Электробритва Series 3 300BT с триммером',
        'nmId': 205064632},
       {'views': 238,
        'clicks': 4,
        'ctr': 1.68,
        'cpc': 4.77,
        'sum': 

In [11]:
def flatten_campaign_data(campaign_data):
    flattened_data = []
    
    for entry in campaign_data:
        advertId = entry.get("advertId")
        for day in entry.get("days", []):
            date = day.get("date")
            for app in day.get("apps", []):
                for nm in app.get("nm", []):
                    flattened_data.append({
                        "date": date,
                        "nmId": nm.get("nmId"),
                        "name": nm.get("name"),
                        "views": nm.get("views"),
                        "clicks": nm.get("clicks"),
                        "ctr": nm.get("ctr"),
                        "cpc": nm.get("cpc"),
                        "sum": nm.get("sum"),
                        "atbs": nm.get("atbs"),
                        "orders": nm.get("orders"),
                        "cr": nm.get("cr"),
                        "shks": nm.get("shks"),
                        "sum_price": nm.get("sum_price"),
                        "advertId": advertId,
                    })
    
    # Create DataFrame
    df = pd.DataFrame(flattened_data)
    
    # Convert and clean date column
    if not df.empty and "date" in df.columns:
        df["date"] = pd.to_datetime(df["date"]).dt.tz_localize(None)
    
    return df

# Process all datasets with one line each
df_guten = flatten_campaign_data(all_campaign_data_guten)
df_giper = flatten_campaign_data(all_campaign_data_giper)
df_kitchen = flatten_campaign_data(all_campaign_data_kitchen)
df_smart = flatten_campaign_data(all_campaign_data_smart)

df_guten


,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-05-27 03:00:00,147720621,Бритва мужская электрическая S5 51-B4650cs с т...,12.0,1.0,8.33,1.23,1.23,0.0,0.0,0.0,0.0,0.0,21202453
1,2025-05-27 03:00:00,147720623,Бритва мужская электрическая S6 61-N1000s,20.0,0.0,0.00,0.00,2.44,0.0,0.0,0.0,0.0,0.0,21202453
2,2025-05-27 03:00:00,213061469,Бритва мужская электрическая S9 9515s,9.0,0.0,0.00,0.00,0.87,0.0,0.0,0.0,0.0,0.0,21202453
3,2025-05-27 03:00:00,203815244,Бритва мужская электрическая S8 8517s с триммером,7.0,0.0,0.00,0.00,0.42,0.0,0.0,0.0,0.0,0.0,21202453
4,2025-05-27 03:00:00,147720619,Бритва мужская электрическая S6 61-R1200s с тр...,61.0,0.0,0.00,0.00,4.98,0.0,0.0,0.0,0.0,0.0,21202453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393,2025-05-27 03:00:00,181674546,Электрическая зубная щётка Pro Series 1,11.0,0.0,0.00,0.00,0.83,0.0,0.0,0.0,0.0,0.0,20139198
394,2025-05-27 03:00:00,64331255,Электрическая зубная щетка взрослая Pro 3 3500...,62.0,3.0,4.84,2.22,6.67,0.0,0.0,0.0,0.0,0.0,20139198
395,2025-05-27 03:00:00,181674546,Электрическая зубная щётка Pro Series 1,33.0,2.0,6.06,1.38,2.76,0.0,0.0,0.0,0.0,0.0,20139198
396,2025-05-27 03:00:00,64331255,Электрическая зубная щетка взрослая Pro 3 3500...,99.0,5.0,5.05,2.09,10.44,0.0,0.0,0.0,0.0,0.0,20139198


In [12]:
def group_and_aggregate(df, project_name):

    grouped_df = (
        df.groupby([df["date"].dt.date, "nmId","advertId"], as_index=False)
        .agg({
            "date": "first",  # Keep the first date (to retain the day)
            "name": "first",  # Keep the first name (or customize this logic)
            "views": "sum",
            "clicks": "sum",
            "ctr": "mean",  # Sum or average, depending on your needs
            "cpc": "mean",  # Use mean for 'cpc' (cost per click)
            "sum": "sum",
            "atbs": "sum",
            "orders": "sum",
            "cr": "mean",  # Use mean for 'cr' (conversion rate)
            "shks": "sum",
            "sum_price": "sum",
            "advertId": "first"  # Keep the first 'advertId' (no summing)
        })
    )
    
    # Add project and marketplace info
    grouped_df['Project'] = project_name
    grouped_df['Marketplace'] = 'Wildberries'
    
    return grouped_df

# Process each DataFrame using the function
df_grouped_guten = group_and_aggregate(df_guten, 'WB-GutenTech')
df_grouped_giper = group_and_aggregate(df_giper, 'WB-ГиперМаркет')
df_grouped_kitchen = group_and_aggregate(df_kitchen, 'WB-KitchenAid')
df_grouped_smart = group_and_aggregate(df_smart, 'WB-Smart-Market')

# Combine all DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)


# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,date,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25088481,2025-05-27 03:00:00,Эпилятор женский электрический Silk-epil 9-880,580.0,38.0,5.520000,4.786667,130.51,4.0,0.0,0.000000,0.0,0.0,21204285,WB-GutenTech,Wildberries
1,25546922,2025-05-27 03:00:00,Эпилятор женский электрический Silk-epil 3-170,0.0,2.0,0.000000,0.000000,0.00,0.0,0.0,0.000000,0.0,0.0,21204098,WB-GutenTech,Wildberries
2,25623718,2025-05-27 03:00:00,Бритва мужская электрическая S3 300BT с триммером,332.0,10.0,2.026667,2.123333,29.87,1.0,0.0,0.000000,0.0,0.0,21202453,WB-GutenTech,Wildberries
3,25623718,2025-05-27 03:00:00,Бритва мужская электрическая S3 300BT с триммером,462.0,21.0,4.276667,4.810000,94.10,2.0,1.0,8.333333,1.0,8480.0,21202745,WB-GutenTech,Wildberries
4,25624340,2025-05-27 03:00:00,Электробритва Series 3 300s,739.0,16.0,1.506667,2.730000,69.05,2.0,0.0,0.000000,0.0,0.0,21202453,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,235490803,2025-05-27 03:00:00,Эпилятор Silk-epil 9 Flex SES 9-011,2.0,0.0,0.000000,0.000000,0.14,2.0,0.0,0.000000,0.0,0.0,20822563,WB-Smart-Market,Wildberries
307,235597032,2025-05-27 03:00:00,Эпилятор женский электрический Silk-epil 7-030,1.0,0.0,0.000000,0.000000,0.07,2.0,0.0,0.000000,0.0,0.0,20822563,WB-Smart-Market,Wildberries
308,235599972,2025-05-27 03:00:00,Электробритва Series 9 Pro+ 9510s,1334.0,33.0,2.406667,4.036667,119.21,3.0,1.0,1.960000,1.0,29977.0,15562363,WB-Smart-Market,Wildberries
309,235604574,2025-05-27 03:00:00,Триммер для бороды и усов BT5421,2.0,1.0,50.000000,0.085000,0.24,0.0,0.0,0.000000,0.0,0.0,19597465,WB-Smart-Market,Wildberries


In [13]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final

,nmId,date,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25088481,2025-05-27 03:00:00,Эпилятор женский электрический Silk-epil 9-880,580.0,38.0,130.51,4.0,0.0,0.0,0.0,21204285,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
1,25546922,2025-05-27 03:00:00,Эпилятор женский электрический Silk-epil 3-170,0.0,2.0,0.00,0.0,0.0,0.0,0.0,21204098,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_1-3 серия Эпиляторы_07.11.2024,Идут показы,Аукцион
2,25623718,2025-05-27 03:00:00,Бритва мужская электрическая S3 300BT с триммером,332.0,10.0,29.87,1.0,0.0,0.0,0.0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25623718,2025-05-27 03:00:00,Бритва мужская электрическая S3 300BT с триммером,462.0,21.0,94.10,2.0,1.0,1.0,8480.0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
4,25624340,2025-05-27 03:00:00,Электробритва Series 3 300s,739.0,16.0,69.05,2.0,0.0,0.0,0.0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,235490803,2025-05-27 03:00:00,Эпилятор Silk-epil 9 Flex SES 9-011,2.0,0.0,0.14,2.0,0.0,0.0,0.0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-28,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
307,235597032,2025-05-27 03:00:00,Эпилятор женский электрический Silk-epil 7-030,1.0,0.0,0.07,2.0,0.0,0.0,0.0,20822563,WB-Smart-Market,Wildberries,2100-01-01,2024-10-22,2025-05-28,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания
308,235599972,2025-05-27 03:00:00,Электробритва Series 9 Pro+ 9510s,1334.0,33.0,119.21,3.0,1.0,1.0,29977.0,15562363,WB-Smart-Market,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания
309,235604574,2025-05-27 03:00:00,Триммер для бороды и усов BT5421,2.0,1.0,0.24,0.0,0.0,0.0,0.0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-05-28,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания


## Статистика карточек товаров по дням
https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history

Response Schema: application/json

- data / Array of objects 
    - nmID / integer <int32> / Артикул W
    - imtName / string / Наименование карточки товар
    - vendorCode/ string/ Артикул продавц
    - history/ Array of objects / Array
        - dt/ string <date> / Дата
        - openCardCount / integer <int32> / Количество переходов в карточку товар
        - addToCartCount / integer <int32> / Положили в корзину, шту
        - ordersCount / integer <int32> / Заказали товаров, ш
        - ordersSumRub / integer <int32> / Заказали на сумму, руб/ buyoutsCount/ integer <int32/ Выкупили товаров, шт
        - buyoutsSumRub / integer <int32> / Выкупили на сумму, руб
        - buyoutPercent / integer <int32> / Процент выкупа, % (Какой процент посетителей, заказавших товар, его выкупили. Без учёта товаров, которые еще доставляются покупателю.)
        - addToCartConversion / numbe / Конверсия в корзину, % (Какой процент посетителей, открывших карточку товара, добавили товар в корзину)
        - cartToOrderConversion / integer <int32> / Конверсия в заказ, % (Какой процент посетителей, добавивших товар в корзину, сделали заказ)
- error/ boolean / Флаг ошибки
- errorText/ string / Описание ошибки
- additionalErrors / Array of objects / Дополнительные ошибки
        - field/ string / Структура, где допущена ошибки
        - description/ string / Описание

In [14]:
# Define headers for each project
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json',
    'Content-Type': 'application/json'  # Ensure this header is set

}

# Define the API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail"

# Set fixed date range for testing
yesterday_start = f'{yesterday} 00:00:00'
yesterday_end = f'{yesterday} 23:59:59'

def get_report(url, headers, begin, end, page, project_name):
    all_data = []
    max_retries = 3
    retry_delay = 30  # seconds
    request_timeout = 60  # seconds
    
    while True:
        for attempt in range(max_retries):
            try:
                # Define the request body with all required parameters
                request_body = {
                    "period": {
                        "begin": begin,
                        "end": end
                    },
                    "orderBy": {
                        "field": "ordersSumRub",
                        "mode": "desc"
                    },
                    "page": page,
                    "timezone": "Europe/Moscow",  # Added as recommended in API docs
                    "brandNames": [],  # Empty array to get all brands
                    "objectIDs": [],   # Empty array to get all objects
                    "nmIDs": []        # Empty array to get all items
                }

                json_data = json.dumps(request_body, ensure_ascii=False)
                
                # Send the POST request with timeout
                response = requests.post(
                    url, 
                    headers=headers, 
                    data=json_data, 
                    timeout=request_timeout
                )
                
                # Handle rate limiting (429 status code)
                if response.status_code == 429:
                    wait_time = int(response.headers.get('Retry-After', retry_delay))
                    print(f"Rate limit exceeded for {project_name}. Waiting {wait_time} seconds...")
                    time.sleep(wait_time)
                    continue
                
                # Check for successful response
                if response.status_code != 200:
                    print(f"Request failed with status code {response.status_code} for {project_name}, page {page}")
                    print("Response text:", response.text)
                    if attempt == max_retries - 1:
                        return all_data
                    time.sleep(retry_delay)
                    continue
                
                data = response.json()
                
                # Validate response structure
                if not data.get('data') or not isinstance(data['data'].get('cards'), list):
                    print(f"Invalid data structure received for {project_name}, page {page}")
                    return all_data
                
                cards = data['data']['cards']
                
                # Check if we have any data
                if not cards:
                    print(f"No more data available for {project_name}")
                    return all_data
                
                all_data.extend(cards)
                print(f"Page {page} retrieved successfully for {project_name} (got {len(cards)} items)")
                
                # Check if there are more pages (using isNextPage flag)
                if not data['data'].get('isNextPage', False):
                    print(f"Reached last page for {project_name}")
                    return all_data
                
                # Prepare for next page
                page += 1
                time.sleep(5)  # Reduced delay between pages to 5 seconds
                break  # Success, break out of retry loop
                
            except requests.exceptions.RequestException as e:
                print(f"Request failed for {project_name}, page {page}: {str(e)}")
                if attempt == max_retries - 1:
                    return all_data
                time.sleep(retry_delay)
                continue
            except Exception as e:
                print(f"Unexpected error for {project_name}, page {page}: {str(e)}")
                return all_data

# Get data for each project
data_guten = get_report(url, headers_guten, yesterday_start, yesterday_end, 1, 'WB-GutenTech')
data_giper = get_report(url, headers_giper, yesterday_start, yesterday_end, 1, 'WB-ГиперМаркет')
data_kitchen = get_report(url, headers_kitchen, yesterday_start, yesterday_end, 1, 'WB-KitchenAid')
data_smart = get_report(url, headers_smart, yesterday_start, yesterday_end, 1, 'WB-Smart-Market')


# Function to flatten the JSON data for the current period
def flatten_json_current_period(cards):
    flattened_data = []
    for card in cards:
        nmID = card["nmID"]
        vendorCode = card["vendorCode"]
        brandName = card["brandName"]
        objectID = card["object"]["id"]
        objectName = card["object"]["name"]
        

        # Extract statistics for the selected period
        selected_period = card["statistics"]["selectedPeriod"]

        flattened_data.append({
            "nmID": nmID,
            "vendorCode": vendorCode,
            "brandName": brandName,
            "objectID": objectID,
            "objectName": objectName,
            "begin": selected_period["begin"],
            "end": selected_period["end"],
            "openCardCount": selected_period["openCardCount"],
            "addToCartCount": selected_period["addToCartCount"],
            "ordersCount": selected_period["ordersCount"],
            "ordersSumRub": selected_period["ordersSumRub"],
            "buyoutsCount": selected_period["buyoutsCount"],
            "buyoutsSumRub": selected_period["buyoutsSumRub"],
            "cancelCount": selected_period["cancelCount"],
            "cancelSumRub": selected_period["cancelSumRub"],
            "avgOrdersCountPerDay": selected_period["avgOrdersCountPerDay"],
            "avgPriceRub": selected_period["avgPriceRub"],
            "addToCartPercent": selected_period["conversions"]["addToCartPercent"],
            "cartToOrderPercent": selected_period["conversions"]["cartToOrderPercent"],
            "buyoutsPercent": selected_period["conversions"]["buyoutsPercent"],
            "stocksMp": card["stocks"]["stocksMp"],
            "stocksWb": card["stocks"]["stocksWb"]
        })

    return flattened_data

# Convert the flattened data to a DataFrame
flattened_data_guten = flatten_json_current_period(data_guten)
flattened_data_giper = flatten_json_current_period(data_giper)
flattened_data_kitchen = flatten_json_current_period(data_kitchen)
flattened_data_smart = flatten_json_current_period(data_smart)

df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)

# Add the 'Project' column to each DataFrame before concatenation
df_guten['Project'] = 'WB-GutenTech'
df_giper['Project'] = 'WB-ГиперМаркет'
df_kitchen['Project'] = 'WB-KitchenAid'
df_smart['Project'] = 'WB-Smart-Market'

# Combine all campaign data
combined_df = pd.concat([df_guten, df_giper, df_kitchen, df_smart], ignore_index=True)
combined_df['Marketplace'] = 'Wildberries'
print("Columns in combined_campaigns:", combined_df.columns.tolist())

# Display the DataFrame
combined_df


# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Page 1 retrieved successfully for WB-GutenTech (got 1000 items)
Page 2 retrieved successfully for WB-GutenTech (got 1000 items)
Page 3 retrieved successfully for WB-GutenTech (got 20 items)
Reached last page for WB-GutenTech
Page 1 retrieved successfully for WB-ГиперМаркет (got 1000 items)
Page 2 retrieved successfully for WB-ГиперМаркет (got 106 items)
Reached last page for WB-ГиперМаркет
Page 1 retrieved successfully for WB-KitchenAid (got 105 items)
Reached last page for WB-KitchenAid
Page 1 retrieved successfully for WB-Smart-Market (got 108 items)
Reached last page for WB-Smart-Market
Columns in combined_campaigns: ['nmID', 'vendorCode', 'brandName', 'objectID', 'objectName', 'begin', 'end', 'openCardCount', 'addToCartCount', 'ordersCount', 'ordersSumRub', 'buyoutsCount', 'buyoutsSumRub', 'cancelCount', 'cancelSumRub', 'avgOrdersCountPerDay', 'avgPriceRub', 'addToCartPercent', 'cartToOrderPercent', 'buyoutsPercent', 'stocksMp', 'stocksWb', 'Project', 'Marketplace']
Combined DataFr

In [15]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [16]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['date'] = pd.to_datetime(df_final_copy['date']).dt.date
df_copy['begin'] = pd.to_datetime(df_copy['begin']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'begin': 'day'}, inplace=True)
df_final_copy.rename(columns={'date': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2



C:\Users\User\AppData\Local\Temp\ipykernel_13292\3298553468.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13292\3298553468.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For e

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25088481,2025-05-27,Эпилятор женский электрический Silk-epil 9-880,580.0,38.0,130.51,4.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,4
1,25546922,2025-05-27,Эпилятор женский электрический Silk-epil 3-170,0.0,2.0,0.00,0.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_1-3 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,0
2,25623718,2025-05-27,Бритва мужская электрическая S3 300BT с триммером,332.0,10.0,29.87,1.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,8480,4
3,25623718,2025-05-27,Бритва мужская электрическая S3 300BT с триммером,462.0,21.0,94.10,2.0,1.0,1.0,8480.0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,1,8480,4
4,25624340,2025-05-27,Электробритва Series 3 300s,739.0,16.0,69.05,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,2,13290,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,235490803,2025-05-27,Эпилятор Silk-epil 9 Flex SES 9-011,2.0,0.0,0.14,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-10-22,2025-05-28,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,1,13461,3
307,235597032,2025-05-27,Эпилятор женский электрический Silk-epil 7-030,1.0,0.0,0.07,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-10-22,2025-05-28,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,7,74466,21
308,235599972,2025-05-27,Электробритва Series 9 Pro+ 9510s,1334.0,33.0,119.21,3.0,1.0,1.0,29977.0,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,2,59954,5
309,235604574,2025-05-27,Триммер для бороды и усов BT5421,2.0,1.0,0.24,0.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-08-29,2025-05-28,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,0,0,2


In [17]:
merged_df_2

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25088481,2025-05-27,Эпилятор женский электрический Silk-epil 9-880,580.0,38.0,130.51,4.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-27,BRAUN_9 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,4
1,25546922,2025-05-27,Эпилятор женский электрический Silk-epil 3-170,0.0,2.0,0.00,0.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_1-3 серия Эпиляторы_07.11.2024,Идут показы,Аукцион,0,0,0
2,25623718,2025-05-27,Бритва мужская электрическая S3 300BT с триммером,332.0,10.0,29.87,1.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,1,8480,4
3,25623718,2025-05-27,Бритва мужская электрическая S3 300BT с триммером,462.0,21.0,94.10,2.0,1.0,1.0,8480.0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,1,8480,4
4,25624340,2025-05-27,Электробритва Series 3 300s,739.0,16.0,69.05,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,2,13290,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306,235490803,2025-05-27,Эпилятор Silk-epil 9 Flex SES 9-011,2.0,0.0,0.14,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-10-22,2025-05-28,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,1,13461,3
307,235597032,2025-05-27,Эпилятор женский электрический Silk-epil 7-030,1.0,0.0,0.07,2.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-10-22,2025-05-28,Braun_Эпиляторы_22.10.2024,Идут показы,Автоматическая кампания,7,74466,21
308,235599972,2025-05-27,Электробритва Series 9 Pro+ 9510s,1334.0,33.0,119.21,3.0,1.0,1.0,29977.0,...,Wildberries,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,Идут показы,Автоматическая кампания,2,59954,5
309,235604574,2025-05-27,Триммер для бороды и усов BT5421,2.0,1.0,0.24,0.0,0.0,0.0,0.0,...,Wildberries,2100-01-01,2024-08-29,2025-05-28,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,0,0,2


## Insert the data

In [18]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-j5ou9lq30ldal602.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                           # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [20]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                       int64
day               datetime64[ns]
name_product              object
views                    float64
clicks                   float64
sum                      float64
atbs                     float64
orders                   float64
shks                     float64
sum_price                float64
advertId                   int64
Project                   object
Marketplace               object
endTime                   object
createTime                object
startTime                 object
name_campaign             object
status                    object
type                      object
ordersCount                int64
ordersSumRub               int64
addToCartCount             int64
dtype: object
Sample data to insert: [(25088481, Timestamp('2025-05-27 00:00:00'), 'Эпилятор женский электрический Silk-epil 9-880', 580.0, 38.0, 130.51, 4.0, 0.0, 0.0, 0.0, 21204285, 'WB-GutenTech', 'Wildberries', datetime.date(2100, 1, 1), datetim